<img src="logo.png" width="10%">

In [1]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import matplotlib.pyplot as plt
from ase.visualize import view

from pyscal.crystal_structures import elements, structures
from pyscal_rdf.structure import StructureGraph

import pyvista as pv

el_list = list(elements.keys())
st_list = list(structures.keys())

In [2]:
el_dropdown = widgets.Dropdown(
    options = el_list,
    value = "Fe",
    description = 'Element',
    disabled=False,
)

el_run_button = widgets.Button(
    description='Generate',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

el_add_to_graph_check = widgets.Checkbox(
    value=True,
    description='Add to graph',
    disabled=False
)

el_repetition_box = widgets.IntText(
    value=1,
    description='Repetitions',
    disabled=False
)

el_out = widgets.Output(layout={'border': '1px solid black'})

def create_element(b=None):
    element = el_dropdown.value
    add_to_graph = el_add_to_graph_check.value
    rep = el_repetition_box.value
    sys = g.create_element(element, add_to_graph=add_to_graph, 
                          repetitions=[rep, rep, rep])
    with el_out:
        pl = pv.Plotter()
        pl.add_points(np.array(sys.atoms.positions), render_points_as_spheres=True,
                     point_size=30);
        box_color = "#90a4ae"
        width=3
        pl.add_lines(np.array([[0,0,0], sys.box[0]]), color=box_color, width=width);
        pl.add_lines(np.array([[0,0,0], sys.box[1]]), color=box_color, width=width);
        pl.add_lines(np.array([[0,0,0], sys.box[2]]), color=box_color, width=width);
        second_apex = [sys.box[0][0], sys.box[1][1], sys.box[2][2]]
        boxdims = sys.box_dimensions
        pl.add_lines(np.array([[0, 0+boxdims[1], 0+boxdims[2]], second_apex]), color=box_color, width=width);
        pl.add_lines(np.array([[0+boxdims[0], 0, 0+boxdims[2]], second_apex]), color=box_color, width=width);
        pl.add_lines(np.array([[0+boxdims[0], 0+boxdims[1], 0], second_apex]), color=box_color, width=width);
        pl.add_lines(np.array([[0, 0, 0+boxdims[2]], [0, 0+boxdims[1], 0+boxdims[2]]]), color=box_color, width=width);
        pl.add_lines(np.array([[0, 0, 0+boxdims[2]], [0+boxdims[0], 0, 0+boxdims[2]]]), color=box_color, width=width);
        pl.add_lines(np.array([[0+boxdims[0], 0, 0+boxdims[2]], [0+boxdims[0], 0, 0]]), color=box_color, width=width);
        pl.add_lines(np.array([[0+boxdims[0], 0+boxdims[1], 0], [0, 0+boxdims[1], 0]]), color=box_color, width=width);
        pl.add_lines(np.array([[0+boxdims[0], 0+boxdims[1], 0], [0+boxdims[0], 0, 0]]), color=box_color, width=width);
        pl.add_lines(np.array([[0, 0+boxdims[1], 0+boxdims[2]], [0, 0+boxdims[1], 0]]), color=box_color, width=width);
        pl.set_background('white')
        pl.show()
    
@el_run_button.on_click
def element_plot_on_click(b):
    create_element()
    
element_tab = VBox(children=[el_dropdown,
                      el_add_to_graph_check,
                      el_repetition_box,
                      el_run_button,
                      el_out])

In [3]:
st_dropdown = widgets.Dropdown(
    options = st_list,
    value = "bcc",
    description = 'Structure',
    disabled=False,
)

st_run_button = widgets.Button(
    description='Generate',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

st_add_to_graph_check = widgets.Checkbox(
    value=True,
    description='Add to graph',
    disabled=False
)

st_repetition_box = widgets.IntText(
    value=1,
    description='Repetitions',
    disabled=False
)

st_lattice_parameter_box = widgets.FloatText(
    value=1,
    description='Lattice Parameter',
    disabled=False
)

st_element_box = widgets.Text(
    value='Fe',
    description='Element',
    disabled=False
)

def create_structure(b=None):
    structure = st_dropdown.value
    add_to_graph = st_add_to_graph_check.value
    rep = st_repetition_box.value
    lattice_parameter = st_lattice_parameter_box.value
    element = st_element_box.value
    sys = g.create_structure(structure, 
                             lattice_constant=lattice_parameter,
                             repetitions=[rep, rep, rep],
                             add_to_graph=add_to_graph,
                             element=element)
    print("done")
    
@st_run_button.on_click
def structure_plot_on_click(b):
    create_structure()
    
structure_tab = VBox(children=[st_dropdown,
                      st_add_to_graph_check,
                      st_repetition_box,
                      st_lattice_parameter_box,
                      st_element_box,
                      st_run_button,])

In [4]:
qt_query_box = widgets.Textarea(
    value="""
        PREFIX cmso: <https://purls.helmholtz-metadaten.de/cmso/>
        SELECT DISTINCT ?symbol
        WHERE {
            ?sample cmso:hasNumberOfAtoms ?number .
            ?sample cmso:hasMaterial ?material .
            ?material cmso:hasStructure ?structure .
            ?structure cmso:hasSpaceGroup ?spacegroup .
            ?spacegroup cmso:hasSpaceGroupSymbol ?symbol .
        FILTER (?number="4"^^xsd:integer)
}""",
    description='Element',
    disabled=False,
    layout=Layout(width="auto", height="100%")
)

out = widgets.Output(layout={'border': '1px solid black'})

qt_run_button = widgets.Button(
    description='Query',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

@qt_run_button.on_click
def query_plot_on_click(b):
    run_query()
    
def run_query(b=None):
    query = qt_query_box.value
    res = g.graph.query(query)
    with out:
        for r in res:
            print(r[0])

query_tab = VBox(children=[qt_query_box,
                      qt_run_button,
                          out])

In [5]:
g = StructureGraph()

In [6]:
#tab2 = VBox(children=[])

tab = widgets.Tab(children=[element_tab, structure_tab, query_tab])
tab.set_title(0, 'create element')
tab.set_title(1, 'create structure')
tab.set_title(2, 'run query')
#VBox(children=[tab, run_button])

tab

In [7]:
outx = widgets.Output(layout={'border': '1px solid black'})
with outx:
    sys = g.create_element("Fe")
    pl = pv.Plotter()
    pl.add_points(np.array(sys.atoms.positions), render_points_as_spheres=True,
                 point_size=30);
    box_color = "#90a4ae"
    width=3
    pl.add_lines(np.array([[0,0,0], sys.box[0]]), color=box_color, width=width);
    pl.add_lines(np.array([[0,0,0], sys.box[1]]), color=box_color, width=width);
    pl.add_lines(np.array([[0,0,0], sys.box[2]]), color=box_color, width=width);
    second_apex = [sys.box[0][0], sys.box[1][1], sys.box[2][2]]
    boxdims = sys.box_dimensions
    pl.add_lines(np.array([[0, 0+boxdims[1], 0+boxdims[2]], second_apex]), color=box_color, width=width);
    pl.add_lines(np.array([[0+boxdims[0], 0, 0+boxdims[2]], second_apex]), color=box_color, width=width);
    pl.add_lines(np.array([[0+boxdims[0], 0+boxdims[1], 0], second_apex]), color=box_color, width=width);
    pl.add_lines(np.array([[0, 0, 0+boxdims[2]], [0, 0+boxdims[1], 0+boxdims[2]]]), color=box_color, width=width);
    pl.add_lines(np.array([[0, 0, 0+boxdims[2]], [0+boxdims[0], 0, 0+boxdims[2]]]), color=box_color, width=width);
    pl.add_lines(np.array([[0+boxdims[0], 0, 0+boxdims[2]], [0+boxdims[0], 0, 0]]), color=box_color, width=width);
    pl.add_lines(np.array([[0+boxdims[0], 0+boxdims[1], 0], [0, 0+boxdims[1], 0]]), color=box_color, width=width);
    pl.add_lines(np.array([[0+boxdims[0], 0+boxdims[1], 0], [0+boxdims[0], 0, 0]]), color=box_color, width=width);
    pl.add_lines(np.array([[0, 0+boxdims[1], 0+boxdims[2]], [0, 0+boxdims[1], 0]]), color=box_color, width=width);
    pl.set_background('white')
    pl.show()

In [8]:
outx

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…